In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Ellipse
import matplotlib.patches as patches
import librosa
from scipy.ndimage import gaussian_filter1d
from math import pi

In [ ]:
#Use for color scale (maybe look for a new one, I just picked this at random)
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan', 'yellowgreen', 'black', 'yellow']

In [ ]:
#returns a dataframe where all rows have at least 1 likelihood value below .9
#Assess how well DLC is labeling data:
def likelihood_check(start, stop, likelihood, dataframe):
    #choose likelihood to look at 
    LT = likelihood
    
    #choose range to look at
    df = dataframe[(dataframe['time_set']>=start) & (dataframe['time_set']<stop)] #not filtered
    
    #Create a dataframe with rows where at least one of the coordinates has likelihood below .9. Can print out to see how much/where DLC is struggling.
    #input different dataframes to assess different time frames 
    likelihood_data = df[(df['nose_likelihood'] < LT) | (df['head_likelihood'] < LT) | (df['earRight_likelihood'] < LT) | (df['earLeft_likelihood'] < LT) | (df['spine1_likelihood'] < LT) | (df['center_likelihood'] < LT) | (df['spine2_likelihood'] < LT) | (df['spine3_likelihood'] < LT)]
    likelihood_minus_nose = df[(df['head_likelihood'] < LT) | (df['earRight_likelihood'] < LT) | (df['earLeft_likelihood'] < LT) | (df['spine1_likelihood'] < LT) | (df['center_likelihood'] < LT) | (df['spine2_likelihood'] < LT) | (df['spine3_likelihood'] < LT)]
    likelihood_center = df[(df['center_likelihood'] < LT)]
    
    return likelihood_data, likelihood_minus_nose, likelihood_center

#reviewed (same as V3/V2)

In [ ]:
#Replace any nose, head or center likelihood below LT as NAn. Interpolate to fill dropped values 
#returns a df with low likelihoods interpolated - be careful of long stretches of NAn (esp with nose)
#RETURNS new df with NaN values interpolated 

def filter_data(threshold, coords):
    LT = threshold  # Threshold value
    
    # Apply filtering row-wise using .loc. Replace with NA
    coords.loc[coords['nose_likelihood'] < LT, ['nose_x', 'nose_y']] = np.nan
    coords.loc[coords['head_likelihood'] < LT, ['head_x', 'head_y']] = np.nan
    coords.loc[coords['center_likelihood'] < LT, ['center_x', 'center_y']] = np.nan
    coords.loc[coords['earRight_likelihood'] < LT, ['earRight_x', 'earRight_y']] = np.nan
    coords.loc[coords['earLeft_likelihood'] < LT, ['earLeft_x', 'earLeft_y']] = np.nan
    coords.loc[coords['spine1_likelihood'] < LT, ['spine1_x', 'spine1_y']] = np.nan
    coords.loc[coords['spine2_likelihood'] < LT, ['spine2_x', 'spine2_y']] = np.nan
    coords.loc[coords['spine3_likelihood'] < LT, ['spine3_x', 'spine3_y']] = np.nan
    
    df = pd.DataFrame()
    
    #perform linear interpolation to fill NaN values (have one df without interpolation if you want to compare)
    df['nose_x'] = coords['nose_x'].interpolate()
    df['nose_y'] = coords['nose_y'].interpolate()
    df['head_x'] = coords['head_x'].interpolate()
    df['head_y'] = coords['head_y'].interpolate()
    df['center_x'] = coords['center_x'].interpolate()
    df['center_y'] = coords['center_y'].interpolate()
    df['earRight_x'] = coords['earRight_x'].interpolate()
    df['earRight_y'] = coords['earRight_y'].interpolate()
    df['earLeft_x'] = coords['earLeft_x'].interpolate()
    df['earLeft_y'] = coords['earLeft_y'].interpolate()
    df['spine1_x'] = coords['spine1_x'].interpolate()
    df['spine1_y'] = coords['spine1_y'].interpolate()
    df['spine2_x'] = coords['spine2_x'].interpolate()
    df['spine2_y'] = coords['spine2_y'].interpolate()
    df['spine3_x'] = coords['spine3_x'].interpolate()
    df['spine3_y'] = coords['spine3_y'].interpolate()    
    
    return df 

"""
def filter_data(threshold, coords):
    LT = threshold  # Threshold value
    
    # Apply filtering row-wise using .loc. Replace with NA
    coords.loc[coords['nose_likelihood'] < LT, ['nose_x', 'nose_y']] = np.nan
    coords.loc[coords['head_likelihood'] < LT, ['head_x', 'head_y']] = np.nan
    coords.loc[coords['center_likelihood'] < LT, ['center_x', 'center_y']] = np.nan
    coords.loc[coords['earRight_likelihood'] < LT, ['earRight_x', 'earRight_y']] = np.nan
    coords.loc[coords['earLeft_likelihood'] < LT, ['earLeft_x', 'earLeft_y']] = np.nan
    coords.loc[coords['spine1_likelihood'] < LT, ['spine1_x', 'spine1_y']] = np.nan
    coords.loc[coords['spine2_likelihood'] < LT, ['spine2_x', 'spine2_y']] = np.nan
    coords.loc[coords['spine3_likelihood'] < LT, ['spine3_x', 'spine3_y']] = np.nan
    
    #this is creating a new, blank dataframe.
    df = pd.DataFrame()
    
    #within the new dataframe, df, the x,y values are included (with interpolation for DLC values below the threshhold)
    #perform linear interpolation to fill NaN values (have one df without interpolation if you want to compare)
    df['nose_x'] = coords['nose_x'].interpolate()
    df['nose_y_negated'] = coords['nose_y'].interpolate()
    df['head_x'] = coords['head_x'].interpolate()
    df['head_y_negated'] = coords['head_y'].interpolate()
    df['center_x'] = coords['center_x'].interpolate()
    df['center_y_negated'] = coords['center_y'].interpolate()
    df['earRight_x'] = coords['earRight_x'].interpolate()
    df['earRight_y_negated'] = coords['earRight_y'].interpolate()
    df['earLeft_x'] = coords['earLeft_x'].interpolate()
    df['earLeft_y_negated'] = coords['earLeft_y'].interpolate()
    df['spine1_x'] = coords['spine1_x'].interpolate()
    df['spine1_y_negated'] = coords['spine1_y'].interpolate()
    df['spine2_x'] = coords['spine2_x'].interpolate()
    df['spine2_y_negated'] = coords['spine2_y'].interpolate()
    df['spine3_x'] = coords['spine3_x'].interpolate()
    df['spine3_y_negated'] = coords['spine3_y'].interpolate()
    
    #DLC negates the y-coordinates, so this reverts them back.
    df['nose_y'] = (-1*(df['nose_y_negated']))
    df['head_y'] = (-1*(df['head_y_negated']))
    df['center_y'] = (-1*(df['center_y_negated']))
    df['earRight_y'] = (-1*(df['earRight_y_negated']))
    df['earLeft_y'] = (-1*(df['earLeft_y_negated']))
    df['spine1_y'] = (-1*(df['spine1_y_negated']))
    df['spine2_y'] = (-1*(df['spine2_y_negated']))
    df['spine3_y'] = (-1*(df['spine3_y_negated']))
    
    return df 

    #reviewed (same as V2/V3)
"""

"\ndef filter_data(threshold, coords):\n    LT = threshold  # Threshold value\n    \n    # Apply filtering row-wise using .loc. Replace with NA\n    coords.loc[coords['nose_likelihood'] < LT, ['nose_x', 'nose_y']] = np.nan\n    coords.loc[coords['head_likelihood'] < LT, ['head_x', 'head_y']] = np.nan\n    coords.loc[coords['center_likelihood'] < LT, ['center_x', 'center_y']] = np.nan\n    coords.loc[coords['earRight_likelihood'] < LT, ['earRight_x', 'earRight_y']] = np.nan\n    coords.loc[coords['earLeft_likelihood'] < LT, ['earLeft_x', 'earLeft_y']] = np.nan\n    coords.loc[coords['spine1_likelihood'] < LT, ['spine1_x', 'spine1_y']] = np.nan\n    coords.loc[coords['spine2_likelihood'] < LT, ['spine2_x', 'spine2_y']] = np.nan\n    coords.loc[coords['spine3_likelihood'] < LT, ['spine3_x', 'spine3_y']] = np.nan\n    \n    #this is creating a new, blank dataframe.\n    df = pd.DataFrame()\n    \n    #within the new dataframe, df, the x,y values are included (with interpolation for DLC val

"\ndef filter_data(threshold, coords):\n    LT = threshold  # Threshold value\n    \n    # Apply filtering row-wise using .loc. Replace with NA\n    coords.loc[coords['nose_likelihood'] < LT, ['nose_x', 'nose_y']] = np.nan\n    coords.loc[coords['head_likelihood'] < LT, ['head_x', 'head_y']] = np.nan\n    coords.loc[coords['center_likelihood'] < LT, ['center_x', 'center_y']] = np.nan\n    coords.loc[coords['earRight_likelihood'] < LT, ['earRight_x', 'earRight_y']] = np.nan\n    coords.loc[coords['earLeft_likelihood'] < LT, ['earLeft_x', 'earLeft_y']] = np.nan\n    coords.loc[coords['spine1_likelihood'] < LT, ['spine1_x', 'spine1_y']] = np.nan\n    coords.loc[coords['spine2_likelihood'] < LT, ['spine2_x', 'spine2_y']] = np.nan\n    coords.loc[coords['spine3_likelihood'] < LT, ['spine3_x', 'spine3_y']] = np.nan\n    \n    #this is creating a new, blank dataframe.\n    df = pd.DataFrame()\n    \n    #within the new dataframe, df, the x,y values are included (with interpolation for DLC val

In [ ]:
#average all of the skeleton points - smoother trajectory than just using the center. Shape is the same as just plotting center
#average for head points, average for upper body - more robust for head angle
#adds new columns to df

def average_points(df):
    #choose columns to average
    all_average_x = ['nose_x', 'head_x', 'center_x', 'earRight_x', 'earLeft_x', 'spine1_x', 'spine2_x', 'spine3_x']
    all_average_y = ['nose_y', 'head_y', 'center_y', 'earRight_y', 'earLeft_y', 'spine1_y', 'spine2_y', 'spine3_y']
    
    #find average row by row
    df['average_x'] = df[all_average_x].mean(axis =1)
    df['average_y'] = df[all_average_y].mean(axis =1)
    
    #average of upper half of body
    to_average_head_x = ['nose_x', 'head_x', 'earRight_x', 'earLeft_x']
    to_average_head_y = ['nose_y', 'head_y', 'earRight_y', 'earLeft_y']
    #note: axis = 1 signifies to calculate the average of each row. axix = 0 can be used to calculate the average of each column.
    df['head_average_x'] = df[to_average_head_x].mean(axis =1)
    df['head_average_y'] = df[to_average_head_y].mean(axis =1)
    
    #average of upper section of body
    to_average_upper_x = ['center_x', 'spine1_x']
    to_average_upper_y = ['center_y', 'spine1_y']
    df['upper_average_x'] = df[to_average_upper_x].mean(axis =1)
    df['upper_average_y'] = df[to_average_upper_y].mean(axis =1)
        
    #reviewed (same as V2/V3)

In [ ]:
#returns total seconds of a timestamp given in H:M:S.ms

#adds a zeroed timestamp to the existing dataframe
#called in convert_time
def total_seconds(x):
    #split up string given as H:M:S.ms
    split_string = x.str.split(':')
    
    #define first break as hour, second as minute, third as seconds. Calculate total seconds from there
    hours = split_string.str[0].astype(int)
    minutes = split_string.str[1].astype(int)
    seconds = split_string.str[2].astype(float)
    return (minutes*60) + (seconds) + (hours*3600)

    #reviewed (same as V2/V3)

In [ ]:
#adds a new time_set column to data frame. stimulus is delivered at 0s
def convert_time(timestamp_path, df, audio_path, i): 
    
    #read in timestamp from bonsai
    timestamp = pd.read_csv(timestamp_path, header =None, names = ['timestamp'])
    
    #convert timestamp to datetime object
    timestamp['date_time'] = pd.to_datetime(timestamp['timestamp'], format = '%Y-%m-%dT%H:%M:%S%z')
    
    #convert to H:M:S.ms
    timestamp['naive_time'] = timestamp['date_time'].dt.strftime('%H:%M:%S.%f')
    
    #convert to seconds and zero first row 
    timestamp['seconds'] = total_seconds(timestamp['naive_time'])
    df['time_set'] = timestamp['seconds'] - time_set_stimulus(df,timestamp, audio_path, i)
    
    #reviewed

In [ ]:
#use moving forward - find time when stimulus begins and call in convert_time function

def time_set_stimulus(df, timestamp, audio_path, i):
    stimulus_time = pd.read_csv(audio_path, header =None, names = ['stimulus_timestamp'])
    
    stimulus_time['date_time'] = pd.to_datetime(stimulus_time['stimulus_timestamp'], format = '%Y-%m-%dT%H:%M:%S%z')

#extract H:M:S
    stimulus_time['naive_time'] = stimulus_time['date_time'].dt.strftime('%H:%M:%S.%f')

#turn time into seconds and zero seconds
    stimulus_time['seconds'] = total_seconds(stimulus_time['naive_time'])
    
    #set stimulus onset as 0s and add to coordinate df
    start = stimulus_time['seconds'].iloc[i-1]
    
    return start

In [ ]:
#ONLY USE WHEN MISSING AUDIO TIMESTAMPS 

#returns start and stop times of stimulus
#input manually identified start and stop times, audio path, and videotimestamp
def audio_timing(f_start, f_end, audio_path, videotimestamp):

    #extract information from audio file - amplitude and samples (wav files are lossless). store in new df
    #len(y) gives the number of samples in audiofile 
    y, sr = librosa.load(audio_path, sr = 44100)  
    df = pd.DataFrame(data = y, columns = ['amplitude'])

    #calculate offset time - subtract the length of the video from the length of the audio:
    offset_value = (len(y)/sr) - videotimestamp
    
    #adjust start of stimulus time to align with the video. This method finds slightly earlier stimulus time than actual
    start = f_start - offset_value
    stop = f_end - offset_value 
    return start, stop

In [ ]:
#create dataframes based on choice of time frame
#RETURNS a new dataframe isolated to chosen time frame
def dataframe_ranges(start_value, stop_value, df):
    
    selected_range = df[(df['time_set']>=start_value) & (df['time_set']<=stop_value)]
    
    return selected_range

    #reviewed (same as V2/V3)

In [ ]:
#convert data into cm and adjust shelter location as if platform center is 0,0
#RETURNS shelter_x, shelter_y, x_diam, y_diam, and df with all coordinates converted

#this is called in single_mouse_data
def convert_data(n, s, e, w, shelter_x_pixel, shelter_y_pixel, df):
    #find diameter in pixels using DLC points 
    x_diam_pixel = abs(e - w)
    y_diam_pixel = abs(n - s)
    
    #actual platform diameter
    platform_diam_cm = 61
    #note:we will need to update theis after changing the platform size.
    
    #find conversion for x and y dimensions 
    pixels_per_cm_x = x_diam_pixel/platform_diam_cm
    pixels_per_cm_y = y_diam_pixel/platform_diam_cm
    
    #calculate platform center from edges
    platform_center_x = w + ((e-w)/2)
    platform_center_y = n + ((s-n)/2)
    
    #convert to cm
    platform_x = platform_center_x/pixels_per_cm_x
    platform_y = platform_center_y/pixels_per_cm_y
    
    #convert and shift shelter so that platform is centered at 0,0
    shelter_x = (shelter_x_pixel/pixels_per_cm_x) - platform_x
    shelter_y = (shelter_y_pixel/pixels_per_cm_y) - platform_y
    
    x_diam = (x_diam_pixel/pixels_per_cm_x)
    y_diam = (y_diam_pixel/pixels_per_cm_y)
    
    #convert the rest of the datapoints in new dataframe - coord_scaled. center platform at 0,0.
    x_columns_to_convert = ['nose_x', 'head_x', 'earRight_x', 'earLeft_x', 'spine1_x', 'spine2_x',  'spine3_x', 'center_x']
    y_columns_to_convert = ['nose_y', 'head_y', 'earRight_y', 'earLeft_y', 'spine1_y', 'spine2_y', 'spine3_y', 'center_y']
    coord_scaled = df.copy()
    coord_scaled[x_columns_to_convert]= (df[x_columns_to_convert]/pixels_per_cm_x) - platform_x
    coord_scaled[y_columns_to_convert]= (df[y_columns_to_convert]/pixels_per_cm_y) - platform_y
    
    return shelter_x, shelter_y, x_diam, y_diam, coord_scaled

In [ ]:
#read in data and get timeframes for analyzing parameters
#returns shelter_x, shelter_y, x_diam, y_diam, new df

#convert all points to cm and center the platform at 0,0. Adjust all points accordingly.
def single_mouse_data(n, s, e, w, shelter_x_pixel, shelter_y_pixel, video_path):
    #read by default 1st sheet of an excel file
    coord = pd.read_csv(video_path)
    
    #call convert_data function (can just combine these)
    shelter_x, shelter_y, x_diam, y_diam, coord_scaled = convert_data(n, s, e, w, shelter_x_pixel, shelter_y_pixel, coord)
    
    return shelter_x, shelter_y, x_diam, y_diam, coord_scaled

In [ ]:
#apply gaussian filter 
#RETURNS new df with filter applied to below columns
def smooth_curve(sigma, dataframe):
    #create empty dataframe
    df = pd.DataFrame()
    
    #define columns of interest 
    displacement = dataframe['displacement']
    speed = dataframe['speed']
    head_angle = dataframe['head_angle']
    head_angle_speed = dataframe['angle_speed']
    
    #apply gaussian filter with chosen sigma and series
    df = dataframe.copy()
    df['displacement'] = gaussian_filter1d(displacement, sigma)
    df['speed'] = gaussian_filter1d(speed, sigma)
    df['head_angle'] = gaussian_filter1d(head_angle, sigma)
    df['angle_speed'] = gaussian_filter1d(head_angle_speed, sigma)
    
    return df

In [ ]:
#calculate displacement of average of points from shelter:
#adds new displacement column to df
def displacement(shelter_x, shelter_y, df):
    x_disp = df['average_x'].to_numpy() - shelter_x
    y_disp = df['average_y'].to_numpy() - shelter_y
    displacement = np.sqrt(x_disp**2 + y_disp**2)
    df['displacement'] = displacement

In [ ]:
#calculate speed of mouse:
#add new column to df for speed and delta_vector (delta_vector to find linearity ratio)
def speed(df):
    #create empty data frame to store information
    speed = pd.DataFrame()

    #calculate change in position and time between rows for x and y respectively
    speed['delta_x'] = df['average_x'].diff()
    speed['delta_y'] = df['average_y'].diff()
    speed['delta_time'] = df['time_set'].diff()

    #calculate speed from distance/time
    speed['delta_vector'] = np.sqrt(speed['delta_x']**2 + speed['delta_y']**2)
    speed['speed'] = speed['delta_vector'] / speed['delta_time']

    #add speed to original data frame:
    df['speed'] = speed['speed']
    
    #also add delta_vector to dataframe for linearity ratio calculation
    df['delta_vector'] = speed['delta_vector']

In [ ]:
#calculate head angle from shelter and add to new column 'head_angle' in df
#arccos is 0 to 180 (consider sign?)

#create new dataframe for intermediate steps
def angle(shelter_x, shelter_y, df):
    angle = pd.DataFrame()

    #vector from head to nose
    angle['delta_x_nose'] = df['head_average_x'] - df['upper_average_x']
    angle['delta_y_nose'] = df['head_average_y'] - df['upper_average_y']
    angle['distance_nose'] = np.sqrt(angle['delta_x_nose']**2 + angle['delta_y_nose']**2)

    #vector from head to center of shelter
    angle['delta_x_shelter'] = shelter_x - df['upper_average_x']
    angle['delta_y_shelter'] = shelter_y - df['upper_average_y']
    angle['distance_shelter'] = np.sqrt(angle['delta_x_shelter']**2 + angle['delta_y_shelter']**2)

    #calculate dot product
    angle['dot'] = (angle['delta_x_nose']) * (angle['delta_x_shelter']) + (angle['delta_y_nose']) * (angle['delta_y_shelter'])

    #calculate head angle from dot product geometry
    angle['head_angle'] = np.arccos(angle['dot'] / (angle['distance_shelter'] * angle['distance_nose']))
    df['head_angle'] = angle['head_angle']

In [ ]:
#speed of head angle change

#same code as velocity escept change in angle over time
def angle_speed(df):
    angle_speed = pd.DataFrame()
   
    angle_speed['delta_angle'] = df['head_angle'].diff()
    angle_speed['delta_time'] = df['time_set'].diff()

    angle_speed['angle_speed'] = angle_speed['delta_angle'] / angle_speed['delta_time']
    df['angle_speed'] = angle_speed['angle_speed']

In [ ]:
#determine region we count as shelter reaching - input through percent
#RETURNS horizontal and vertical lines defining shelter region 
def shelter_extension_border(shelter_x, shelter_y, percent):
    #based on where the shelter is located/how it is oriented, extend by chosen percent 
    #remember DLC inverts the y axis
    
    if shelter_x < -15 or shelter_x >15:
        shelter_x_extension = 6.25* (1 + (percent / 100))
        shelter_y_extension = 5.25* (1 + (percent / 100))
        
        if shelter_x<-15:
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_y_top = shelter_y - shelter_y_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
    
        elif shelter_x>15:
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_y_top = shelter_y - shelter_y_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
    
    else:
        shelter_x_extension = 5.25* (1 + (percent / 100))
        shelter_y_extension = 6.25* (1 + (percent / 100))
        
        if shelter_y>15:
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
            shelter_y_top = shelter_y - shelter_y_extension
            
        elif shelter_y<15:
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_y_top = shelter_y - shelter_y_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
            
    
    return shelter_x_left, shelter_x_right, shelter_y_top, shelter_y_bottom

In [ ]:
#same process but to find the border of the shelter alone
#RETURNS horizontal and vertical lines defining the shelter region 
def shelter_border(shelter_x, shelter_y):
    if shelter_x < -15 or shelter_x >15:
        shelter_x_extension = 6.25
        shelter_y_extension = 5.25
        
        if shelter_x<-15:
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_y_top = shelter_y - shelter_y_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
    
        elif shelter_x>15:
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_y_top = shelter_y - shelter_y_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
    
    else:
        shelter_x_extension = 5.25
        shelter_y_extension = 6.25
        
        if shelter_y>15:
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
            shelter_y_top = shelter_y - shelter_y_extension
            
        elif shelter_y<15:
            shelter_x_left = shelter_x - shelter_x_extension
            shelter_x_right = shelter_x + shelter_x_extension
            shelter_y_top = shelter_y - shelter_y_extension
            shelter_y_bottom = shelter_y + shelter_y_extension
            
    
    return shelter_x_left, shelter_x_right, shelter_y_top, shelter_y_bottom

In [ ]:
#stop value should be more precise escape done. Percent is how much of shelter extension we want
#RETURNS true or false for mouse escape 
def escape_response(stop_value, shelter_x, shelter_y, df, percent, time, linearity):
    escape = None
    #determine if mouse reaches shelter
    #establish dataframe with timerange of interest
    selected_range = dataframe_ranges(0, stop_value, df)
    
    #determin border for shelter extension range
    shelter_x_left, shelter_x_right, shelter_y_top, shelter_y_bottom = shelter_extension_border(shelter_x, shelter_y, percent)
    
    #does mouse ever enter the extension range
    to_shelter = ((selected_range['average_x'] > shelter_x_left) & (selected_range['average_x'] < shelter_x_right) & (selected_range['average_y'] > shelter_y_top) & (selected_range['average_y'] < shelter_y_bottom)).any()
    
    #determine if mouse reaches shelter within 12s (adjust this time frame)
    first_entry_time = 20
    if to_shelter: 
        shelter_reach = ((selected_range['average_x'] > shelter_x_left) & (selected_range['average_x'] < shelter_x_right) & (selected_range['average_y'] > shelter_y_top) & (selected_range['average_y'] < shelter_y_bottom))
        entries = selected_range[shelter_reach]
        first_entry_time = entries['time_set'].iloc[0]
    
    if first_entry_time <=time:
        escape_time = True
    else:
        escape_time = False
        
    #determine if mouse reaches shelter within certain linearity parameter (adjust this parameter too)
    if ratio < linearity:
        linearity = True
    else:
        linearity = False
        
    if to_shelter & escape_time & linearity:
        escape = True
    else:
        escape = False
        
    return escape

In [ ]:
#Change this so that we press a button when the mouse reaches the shelter and this value is stored. 

#determine what time to terminate escape trajectory at
#RETURNS time when mouse reaches or is closest to the shelter

def escape_timeframe(shelter_x, shelter_y, stop_value, df):
    
    #determine boundary of shelter:
    shelter_x_left, shelter_x_right, shelter_y_top, shelter_y_bottom = shelter_border(shelter_x, shelter_y)
    shelter_x_left_ext, shelter_x_right_ext, shelter_y_top_ext, shelter_y_bottom_ext = shelter_extension_border(shelter_x, shelter_y, 30)
    
    #isolate timeframe of interest from stimulus onset to escape_estimate 
    selected_range = dataframe_ranges(0, stop_value, df)
    
    #escape stores
    escape = ((selected_range['average_x'] > shelter_x_left) & (selected_range['average_x'] < shelter_x_right) & (selected_range['average_y'] > shelter_y_top) & (selected_range['average_y'] < shelter_y_bottom)).any()
    border = ((selected_range['average_x'] > shelter_x_left_ext) & (selected_range['average_x'] < shelter_x_right_ext) & (selected_range['average_y'] > shelter_y_top_ext) & (selected_range['average_y'] < shelter_y_bottom_ext)).any()

    if escape:
        
        shelter_reach = ((selected_range['average_x'] > shelter_x_left) & (selected_range['average_x'] < shelter_x_right) & (selected_range['average_y'] > shelter_y_top) & (selected_range['average_y'] < shelter_y_bottom))
        entries = selected_range[shelter_reach]
        closest_time_shelter = entries['time_set'].iloc[0]
       
        shelter_extension_reach = ((selected_range['average_x'] > shelter_x_left_ext) & (selected_range['average_x'] < shelter_x_right_ext) & (selected_range['average_y'] > shelter_y_top_ext) & (selected_range['average_y'] < shelter_y_bottom_ext))
        entries = selected_range[shelter_extension_reach]
        closest_time_extension = entries['time_set'].iloc[0]
        ratio,initial_displacement, total_distance = linearity_ratio(closest_time_extension, closest_time_shelter,coord_scaled)

        if ratio < 1.3:
            closest_time = closest_time_shelter
            
        else:
            closest_time = closest_time_extension
            
    elif border:
        shelter_extension_reach = ((selected_range['average_x'] > shelter_x_left_ext) & (selected_range['average_x'] < shelter_x_right_ext) & (selected_range['average_y'] > shelter_y_top_ext) & (selected_range['average_y'] < shelter_y_bottom_ext))
        entries = selected_range[shelter_extension_reach]
        closest_time = entries['time_set'].iloc[0] 
        
    else:
        closest_distance = selected_range['displacement'].min()
        closest_distance_index = selected_range['displacement'].idxmin()
        closest_time = selected_range['time_set'].loc[closest_distance_index]
        
    return closest_time


In [ ]:
#plot trajectory for initial 2 minutes of phase 2 video
def before_shelter(ax, shelter_x, shelter_y, x_diam, y_diam, dataframe):

    start_value = dataframe['time_set'].iloc[0]
    stop_value = start_value + 120
    
    df = dataframe_ranges(start_value, stop_value, dataframe)

    #make x and y scales equal - present as a circle
    ax.axis('equal')
    
    #plot the average x and y coordinates
    x = df['average_x']
    y = df['average_y']
    

    #determine the platform and shelter
    platform = Ellipse(xy = (0, 0), width = x_diam, height = y_diam, edgecolor = 'black', facecolor = 'none', alpha = .5, linewidth = .8)

    #plot
    ax.plot(x,y, color = 'blue', linewidth = 1)
    ax.add_patch(platform)
    #add an arrow to show mouse's direction
    #add an arrow to show mouse's direction
    xf = df['average_x'].iloc[-1] #final x coordinate
    xo = df['average_x'].iloc[-2] #second to last x coordinate

    yf = df['average_y'].iloc[-1] #final y coordinate
    yo = df['average_y'].iloc[-2] #second to last y coordinate

    #plot arrow: base x, base y, displacement x, displacement y
    ax.arrow(xo, yo, xf-xo, yf-yo, head_width = 1, head_length = 1, fc = 'blue', ec = "none")
    ax.set_title('2 minute trajectory before shelter exposure')  #would be after stimulus

    #remove axes and box
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
        
    return plt

In [ ]:
#plot trajectory using average skeleton - from chosen start value (ex: -60) to escape end
def before_escape(ax, shelter_x, shelter_y, x_diam, y_diam, start_value, stop_value, dataframe):

    df = dataframe_ranges(start_value, .04, dataframe)
    escape_df = dataframe_ranges(0, stop_value, dataframe)

    #make x and y scales equal - present as a circle
    ax.axis('equal')

    
    #plot the average x and y coordinates
    x1 = df['average_x']
    y1 = df['average_y']
    
    x2 = escape_df['average_x']
    y2 = escape_df['average_y']
    

    #determine the platform and shelter
    platform = Ellipse(xy = (0, 0), width = x_diam, height = y_diam, edgecolor = 'black', facecolor = 'none', alpha = .5, linewidth = .8)

    #determine orientation of platform based on platform location 
    if shelter_x<-15 or shelter_x>15:
        shelter = patches.Rectangle(((shelter_x - 6.25), (shelter_y-5.25)), 12.5,10.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1)
        
    else:
        shelter = patches.Rectangle(((shelter_x - 5.25), (shelter_y-6.25)), 10.5,12.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1)
        

    #plot
    ax.plot(x1,y1, color = 'blue', linewidth = 1)
    ax.plot(x2,y2, color = 'red', linewidth = 1)
    ax.add_patch(platform)
    ax.add_patch(shelter)
    ax.text(shelter_x-1,shelter_y-1, 'S', color = 'black', size = 10)

    #add an arrow to show mouse's direction
    #add an arrow to show mouse's direction
    xf = escape_df['average_x'].iloc[-1] #final x coordinate
    xo = escape_df['average_x'].iloc[-2] #second to last x coordinate

    yf = escape_df['average_y'].iloc[-1] #final y coordinate
    yo = escape_df['average_y'].iloc[-2] #second to last y coordinate

    #plot arrow: base x, base y, displacement x, displacement y
    ax.arrow(xo, yo, xf-xo, yf-yo, head_width = 1, head_length = 1, fc = 'red', ec = "none")
    ax.set_title('Trajectory after shelter removal untill stimulus')  #would be after stimulus

    #remove axes and box
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
        
    return plt

In [ ]:
#plot mouse before shelter is placed, and a minute before stimulus is given
def before_shelter_trajectory(shelter_x, shelter_y, x_diam, y_diam, stop_value, df):
    
    #figure with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    #plot each trajectory side by side
    before_shelter(ax1, shelter_x, shelter_y, x_diam, y_diam, df)
    before_escape(ax2, shelter_x, shelter_y, x_diam, y_diam, -100, stop_value, df) 

    
    #set aspect ratio for axes as equal (size varies right now because head arrows extend of platform)
    for ax in [ax1, ax2]:
        ax.set_aspect('equal')
    
    return plt

In [ ]:
#plot trajectory using average of mouse position.
def trajectory(ax, shelter_x, shelter_y, x_diam, y_diam, start_value, stop_value, dataframe):

    #the dataframe used here is coord_scaled.
    df = dataframe_ranges(start_value, stop_value, dataframe)

    #the below code ensures that the x and y axis scales are the same, ensuring that data shows up as a circle.
    ax.axis('equal')

    #plot the average x and y coordinates 
    x = df['average_x']
    y = df['average_y']

    #determine the platform and shelter
    #note: the Ellipse function was imported
    platform = Ellipse(xy = (0, 0), width = x_diam, height = y_diam, edgecolor = 'black', facecolor = 'none', alpha = .5, linewidth = .8)

    #determine orientation of platform based on platform location 
    if shelter_x<-15 or shelter_x>15:
        shelter = patches.Rectangle(((shelter_x - 6.25), (shelter_y-5.25)), 12.5,10.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1, label = 'shelter')
        shelter_extension = patches.Circle((shelter_x, shelter_y), 10.21, color='green', alpha=0.2, zorder = 1, label = 'Successful Escape')
        #switch with this is escape area is modeled as a rectangle... but for now, using a circle b/c it likely does not need to be/maybe shouldn't be percise. patches.Rectangle(((shelter_x - 6.25*1.3), (shelter_y-5.25*1.3)), 12.5*1.3,10.5*1.3,  linewidth = .8, edgecolor = 'none', facecolor = '#4682B4', alpha = .1,  label = 'Successful Escape')
    else:
        shelter = patches.Rectangle(((shelter_x - 5.25), (shelter_y-6.25)), 10.5, 12.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1, label = 'shelter')
        shelter_extension = patches.Circle((shelter_x, shelter_y), 10.21, color='green', alpha=0.2, zorder = 1, label = 'Successful Escape')
        #Rectangle escape area... Rectangle(((shelter_x - 5.25*1.3), (shelter_y-6.25*1.3)), 10.5*1.3,12.5*1.3,  linewidth = .8, edgecolor = 'none', facecolor = '#4682B4', alpha = .1,  label = 'Successful Escape')
        #note: the 10.21 above is 1.25*radius of shelter (meaning distance from center to far corner)
    #plot
    ax.plot(x,y, color = 'blue', linewidth = 1)
    #the below codes adds the shelter and platform to the visual
    ax.add_patch(platform)
    ax.add_patch(shelter)
    #this adds an S onto the shelter visual
    ax.text(shelter_x-1,shelter_y-1, 'S', color = 'black', size = 10)

    #add an arrow to show mouse's direction
    #add an arrow to show mouse's direction
    xf = df['average_x'].iloc[-1] #final x coordinate
    xo = df['average_x'].iloc[-2] #second to last x coordinate
    yf = df['average_y'].iloc[-1] #final y coordinate
    yo = df['average_y'].iloc[-2] #second to last y coordinate

    #plot arrow: base x, base y, displacement x, displacement y
    ax.arrow(xo, yo, xf-xo, yf-yo, head_width = 1, head_length = 1, fc = 'blue', ec = "none")
    ax.set_title('Trajectory after stimulus onset')

    #remove axes and box (for better visual)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    #Shelter_extension is to model the escape location of the mouse.
    ax.add_patch(shelter_extension)
    ax.legend(loc='upper right', fontsize='small')

    ax.set_xticks([])
    ax.set_yticks([])
        
    return plt

    #reviewed

In [ ]:
#create a new data table with data needed for arrows
#same as plain trajectory but with head direction
def head_angle_trajectory_figure(ax, shelter_x, shelter_y, x_diam, y_diam, start_value, stop_value, dataframe):

    df = dataframe_ranges(start_value, stop_value, dataframe)
    #make x and y scales equal - present as a circle
    ax.axis('equal')

    #look at average x and y coord for trajectory
    x = df['average_x']
    y = df['average_y']
    

    #determine the platform and shelter
    platform = Ellipse(xy = (0, 0), width = x_diam, height = y_diam, edgecolor = 'black', facecolor = 'none', alpha = .5, linewidth = .8)

    #determine orientation of platform based on platform location 
    if shelter_x<-15 or shelter_x>15:
        shelter = patches.Rectangle(((shelter_x - 6.25), (shelter_y-5.25)), 12.5,10.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1, label = 'shelter')
        shelter_extension = patches.Circle((shelter_x, shelter_y), 10.21, color='green', alpha=0.2, zorder = 1, label = 'Successful Escape')
        #shelter_extension = patches.Rectangle(((shelter_x - 6.25*1.3), (shelter_y-5.25*1.3)), 12.5*1.3,10.5*1.3,  linewidth = .8, edgecolor = 'none', facecolor = '#4682B4', alpha = .1,  label = 'Succesful Escape')
    else:
        shelter = patches.Rectangle(((shelter_x - 5.25), (shelter_y-6.25)), 10.5,12.5,  linewidth = .8, edgecolor = 'none', facecolor = 'blue', alpha = .1, label = 'shelter')
        shelter_extension = patches.Circle((shelter_x, shelter_y), 10.21, color='green', alpha=0.2, zorder = 1, label = 'Successful Escape')
        #shelter_extension = patches.Rectangle(((shelter_x - 5.25*1.3), (shelter_y-6.25*1.3)), 10.5*1.3,12.5*1.3,  linewidth = .8, edgecolor = 'none', facecolor = '#4682B4', alpha = .1,  label = 'Succesful Escape')
        
   
    ax.text(shelter_x-1,shelter_y-1, 'S', color = 'black', size = 10)

    
    ax.plot(x,y, color = 'gray', linewidth = 1)
    
    
    #create new dataframe for arrow calculation
    arrow = pd.DataFrame()
    arrow['arrow_base_x'] = df['average_x']
    arrow['arrow_base_y'] = df['average_y']
    arrow['arrow_delta_x'] = df['head_average_x'] - df['upper_average_x']
    arrow['arrow_delta_y'] = df['head_average_y'] - df['upper_average_y']
    arrow['delta_x_nose'] = df['head_average_x'] - df['upper_average_x']
    arrow['delta_y_nose'] = df['head_average_y'] - df['upper_average_y']

    for x,y,dx,dy in zip(arrow['arrow_base_x'], arrow['arrow_base_y'], arrow['arrow_delta_x'], arrow['arrow_delta_y']):
        ax.arrow(x,y,dx*2,dy*2, width = .07, head_width = 1, head_length = 1, overhang =.7, fc = 'b', ec = 'none', length_includes_head =True)

    #plot all else and remove axes
    ax.add_patch(platform)
    ax.add_patch(shelter)
    ax.add_patch(shelter_extension)
    ax.set_title('Trajectory and head angle after stimulus onset')
    ax.legend(loc='upper right', fontsize='small')

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

In [ ]:
def trajectories(shelter_x, shelter_y, x_diam, y_diam, start_value, stop_value, df):
    #figure with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    #plot each trajectory side by side 
    trajectory(ax1, shelter_x, shelter_y, x_diam, y_diam, start_value, stop_value, df)
    head_angle_trajectory_figure(ax2, shelter_x, shelter_y, x_diam, y_diam, start_value, stop_value, df)
    
    #set aspect ratio for axes as equal (size varies right now because head arrows extend of platform)
    for ax in [ax1, ax2]:
        ax.set_aspect('equal')
        
    return plt

In [ ]:
#plot speed vs time
#input choice of start and stop time, dataframe, whether to apply gaussian filter, overlay of filter on raw, sigma value
#RETURNS plot of speed vs time
def speed_figure(ax, start_value, stop_value, dataframe, smooth, overlay, aligned, sigma):

    df = dataframe_ranges(start_value, stop_value, dataframe)
    
    x = df['time_set']
    y = df['speed']
    y_smooth = gaussian_filter1d(y, sigma)

    #choose whether to look at filtered, raw, or overlay
    if smooth:
        ax.plot(x,y_smooth, color = (0,0,1,.8)) 
        
    elif overlay:
        ax.plot(x,y_smooth, color = (0,0,1,.8))
        ax.plot(x,y, color = (.5,.5,.5,.6))
        
    else:
        ax.plot(x,y, color = (0,0,1,.8))    
        
    #if lining up vertically, remove tick marks and x axis label    
    if aligned:
        ax.set_ylabel('Speed (cm/s)')
        ax.xaxis.set_ticks([])        
        
    else:   
        ax.set_xlabel('Time from stimulus onset (s)')
        ax.set_ylabel('Speed (cm/s)') #convert to cm - cm/s
        ax.set_title('Speed after stimulus onset')


    #whether stimulus is represented by dotted line or shaded region (if region longer than stimulus time)
    if (stop_value)<7.5:
        ax.axvline(x=0, color='gray', linestyle='--', linewidth=1, label='Vertical Line')
    else:
        ax.axvspan(0, 7.5, color = 'b', alpha =.08)
        
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    return plt

In [ ]:
#same as speed
#RETURNS displacement vs time
def displacement_figure(ax, start_value, stop_value, dataframe, boolean, overlay, aligned, sigma):
    #graph of displacement vs time after stimulus plt.figure() ax = plt.axes()
    df = dataframe_ranges(start_value, stop_value, dataframe)

    x = df['time_set']
    y = df['displacement']
    y_smooth = gaussian_filter1d(y, sigma)

    
    if boolean:
        ax.plot(x,y_smooth, color = (0,0,1,.8))        
    elif overlay:
        ax.plot(x,y_smooth, color = (0,0,1,.8))
        ax.plot(x,y, color = (.5,.5,.5,.6))
    else:
        ax.plot(x,y, color = (0,0,1,.8))
    
    #use if aligning speed, displacement, and head angle vertically. Remove x axis ticks
    if aligned:
        ax.set_ylabel('Displacement from shelter (cm)') 
        ax.xaxis.set_ticks([])

    #otherwise, use these labels
    else:
        ax.set_xlabel('Time from stimulus onset (s)')
        ax.set_ylabel('Displacement from shelter (cm)') 
        ax.set_title('Displacement from shelter after stimulus onset')


    if (stop_value)<7.5:
        ax.axvline(x=0, color='gray', linestyle='--', linewidth=1, label='Vertical Line')
    else:
        ax.axvspan(0, 7.5, color = 'b', alpha =.08)
 
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Define y-axis boundaries for shading
    y_min_shade = 0
    y_max_shade = 8.16

    # Fill the entire background from y_min_shade to y_max_shade
    plt.fill_between(x, y_min_shade, y_max_shade, color='gray', alpha=0.4)
    
    return plt

In [ ]:
#same as displacement and speed
#RETURNS plot of head angle vs time
def head_angle_figure(ax, start_value, stop_value, dataframe, boolean, overlay, aligned, sigma):
    df = dataframe_ranges(start_value, stop_value, dataframe)

    x = df['time_set']
    y = df['head_angle']
    y_smooth = gaussian_filter1d(y, sigma)

    
    if boolean:
        ax.plot(x,y_smooth, color = (0,0,1,.8))        
    elif overlay:
        ax.plot(x,y_smooth, color = (0,0,1,.8))
        ax.plot(x,y, color = (.5,.5,.5,.6))
    else:
        ax.plot(x,y, color = (0,0,1,.8))        
     
    
    if aligned:
        ax.set_ylabel('Head angle from shelter (radians)')
        ax.set_xlabel('Time from stimulus onset (s)')


    else:
        ax.set_xlabel('Time from stimulus onset (s)')
        ax.set_ylabel('Head angle from shelter (radians)')
        ax.set_title('Head angle after stimulus presentation')

    if (stop_value)<7.5:
        ax.axvline(x=0, color='gray', linestyle='--', linewidth=1, label='Vertical Line')
    else:
        ax.axvspan(0, 7.5, color = 'b', alpha =.08)
   
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    return plt

In [ ]:
#define initial displacement as initial linear distance of mouse from first to last point; total distance is distance mouse actually travels
#only calculate for succesful shelter reaching mice - use escape only times
#RETURNS linearity ratio (initial displacement/total distance), initial displacement, and total distance 
def linearity_ratio(start_value,stop_value, dataframe):
    df = dataframe_ranges(start_value, stop_value, dataframe)
           
    total_distance = np.sum(df['delta_vector'])
    #calculate initial linear displacement from ending point 
    displacement_x = df['average_x'].iloc[0] - df['average_x'].iloc[-1] #last x coordinate of mouse in time frame
    displacement_y = df['average_y'].iloc[0] - df['average_y'].iloc[-1] #last y coordinate of mouse in time frame 
    initial_displacement = np.sqrt(displacement_x**2 + displacement_y**2)

    #find ratio - total distance traveled / initial displacement -- smaller is more linear
    ratio = total_distance/initial_displacement
    
    return ratio, initial_displacement, total_distance

In [ ]:
#rotates trajectories so that the shelter is straight vertical and centered at 0,25
def align_shelter(shelter_x, shelter_y, start_value, stop_value, dataframe):
    
    rotated = pd.DataFrame()
    shelter_rotated = pd.DataFrame()
    
    #select timeframe to look at 
    df = dataframe_ranges(start_value, stop_value, dataframe)
    
    #store x and y values as x and y arrays
    x = df['average_x'].values
    y = df['average_y'].values
    
    angle = 0

    #rotation angle based on where the shelter is - placement along 4 quadrants is fairly consistent
    if shelter_x < -15: #on left side
        angle = -pi/2
    if shelter_x > 15: #on right side
        angle = pi/2
    if shelter_y < -15: #bottom
        angle = pi 
        

    #rotate vector with rotation matrix:
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)
    rotation_matrix = np.array([[cos_theta, -sin_theta], [sin_theta, cos_theta]])
    coords = np.vstack((x, y))
    shelter_coords = np.vstack((shelter_x, shelter_y))
   
    rotated_coords = rotation_matrix @ coords
    rotated_shelter_coords = rotation_matrix @ shelter_coords
    
    
    #store the rotated coordinates in new/empty dfs
    rotated['rotated_x'] = rotated_coords[0] 
    rotated['rotated_y'] = rotated_coords[1]

    shelter_rotated_x = rotated_shelter_coords[0]
    shelter_rotated_y = rotated_shelter_coords[1]

    #shift value:
    x_shift_value = - shelter_rotated_x
    y_shift_value = 25 - shelter_rotated_y
    
    #set the shelter at 0,25
    shelter_shift_x = 0 
    shelter_shift_y = 25
    
    rotated['rotated_shift_x'] = rotated['rotated_x'] + x_shift_value
    rotated['rotated__shift_y'] = rotated['rotated_y'] + y_shift_value
    
    return rotated, shelter_shift_x, shelter_shift_y

In [ ]:
#no longer graphing normalized values (kept as a comment). We already graphed everything needed, so here we are just overlaying them.

def combined_graphs(start_value, stop_value, dataframe):
    #data for graph is below
    #to be graphed on left axis
    df = dataframe_ranges(start_value, stop_value, dataframe)
    x = df['time_set']
    y_disp = df['displacement']
    y_speed = df['speed']
    #graph on right axis
    pi = math.pi
    y_angle = (180/(pi))*(df['head_angle'])
    #y_angle = df['head_angle']
    
    """
    y_disp = normalize(df['displacement'])
    y_speed = normalize(df['speed'])
    y_angle = normalize(df['head_angle'])
    """
    fig, ax1 = plt.subplots()
    #ax=plt.axes()
     # Define y-axis boundaries for shading
    y_min_shade = 0
    y_max_shade = 8.16

    # Fill the entire background from y_min_shade to y_max_shade
    plt.fill_between(x, y_min_shade, y_max_shade, color='gray', alpha=0.4)
    
    #left axis
    ax1.plot(x, y_disp, color = 'blue', label = 'displacement')
    ax1.plot(x, y_speed, color = 'red', label = 'speed')
    ax1.set_xlabel('Time from stimulus onset (s)')
    ax1.set_ylabel('Displacement (cm) and speed (cm/s)')
    
    #right axis
    ax2 = ax1.twinx()
    ax2.plot(x, y_angle, color = 'green', label = 'head angle')
    ax2.set_ylabel('Head angle from shelter (degrees)')
    
    if stop_value<8:
        plt.axvline(x=0, color='gray', linestyle='--', linewidth=1)
    else:
        plt.axvspan(0, stop_value, color = 'b', alpha =.08)
   
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
        
    # Collect handles and labels from both axes
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()

    # Combine handles and labels
    handles = handles1 + handles2
    labels = labels1 + labels2

    # Create a unified legend
    fig.legend(handles, labels, loc='upper right', fontsize='small')

    #ax1.legend(loc = 'upper right', fontsize = 'small')
    #ax2.legend(loc = 'upper right', fontsize = 'small')
    
    plt.title('Displacement, speed, and head angle')
    
    return plt

    #reviewed

In [ ]:
#NEEDS WORK/REVIEW, did not review
#average angle change in head angle speed

#head angle vs displacement from shelter
#average angle change in head angle speed

#head angle vs displacement from shelter
def angle_speed_figure(ax, stop, length, df):
    plt.figure()
    ax = plt.axes()

    x = df['time_set']
    y = df['angle_speed']

    ax.plot(x,y)
    ax.set_xlabel('Time from stimulus (s)')
    ax.set_ylabel('Speed of head angle change from shelter')
    ax.set_title('Speed of head angle change after stimulus onset')

    ax.axhline(y=0, color = 'gray', linestyle = '--')

    if length<8:
        plt.axvline(x=0, color='gray', linestyle='--', linewidth=1, label='Vertical Line')
    else:
        plt.axvspan(0, stop, color = 'b', alpha =.08)

    ax.text(.5,.0, 'Toward shelter', ha = 'center', va = 'bottom', transform = ax.transAxes) 
    ax.text(.5,.93, 'Away from shelter', ha = 'center', va = 'bottom', transform = ax.transAxes) 

    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)

    return plt

In [ ]:
#return normalized df
def normalize(df):
    min_value = df.min()
    max_value = df.max()
    range_value = max_value - min_value
    normalized = (df - min_value) / range_value
    return normalized

In [ ]:
#NOT FUNCTIONAL RIGHT NOW - trying to replicate graph from rapid spatial learning paper where shelter is aligned vertically and distance is normalized
def rotate_trajectory(shelter_x, shelter_y, start_value, stop_value, dataframe):
    rotated = pd.DataFrame()
    shelter_rotated = pd.DataFrame()
    
    df = dataframe_ranges(start_value, stop_value, dataframe)
    
    #store x and y values as x and y arrays
    x = df['center_x'].values
    y = df['center_y'].values
    
    #initial x and y values to recenter the data:
    initial_x = x[0]
    initial_y = y[0]
    
    #set the start of stimulus location as 0,0
    x_shift = x - initial_x
    y_shift = y - initial_y
    shelter_x_shift = shelter_x - initial_x
    shelter_y_shift = shelter_y - initial_y
    
    #shift the shelter too:
    shelter_x_shift = shelter_x - initial_x
    shelter_y_shift = shelter_y - initial_y
    
    
    #rotate the trajectory so that shelter is on vertical 
    #find angle between vector to shelter and vector to vertical. Rotate by that angle 
    delta_x_shelter = shelter_x - initial_x
    delta_y_shelter = shelter_y - initial_y
    vector_shelter = np.sqrt(delta_x_shelter**2 + delta_y_shelter**2)
    
    delta_x_point = 0 #same initial x
    delta_y_point = 10   #initial y plus 10 in vertical
    vector_point = np.sqrt(delta_x_point**2 + delta_y_point**2)
    
    
    #find the angle between the two points with cross product: 
    dot = delta_x_shelter * delta_x_point + delta_y_shelter * delta_y_point
    angle = np.arccos(dot / (vector_shelter * vector_point))
    
    if shelter_x_shift<0:
        angle = -angle  
    
    #rotate vector with rotation matrix:
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)
    rotation_matrix = np.array([[cos_theta, -sin_theta], [sin_theta, cos_theta]])
    coords = np.vstack((x_shift, y_shift))
    shelter_coords = np.vstack((shelter_x_shift, shelter_y_shift))
   
    rotated_coords = rotation_matrix @ coords
    rotated_shelter_coords = rotation_matrix @ shelter_coords
    
    rotated['rotated_x'] = rotated_coords[0]
    rotated['rotated_y'] = rotated_coords[1]

    shelter_rotated_x = rotated_shelter_coords[0]
    shelter_rotated_y = rotated_shelter_coords[1]

    
    return rotated, shelter_rotated_x, shelter_rotated_y

In [ ]:
#also not currently using 
def normalize_rotation(rotate, shelter_rotate_x, shelter_rotate_y):
    normalized = pd.DataFrame()
    
    #these are all rotated but not normalized values 
    
    x = rotate['rotated_x']
    y = rotate['rotated_y']
    shelter_x = shelter_rotate_x
    shelter_y = shelter_rotate_y
    
    #add the shelter coordinates to the x and y arrays - include in the normalization range. Shelter should be at 0, 1
    x_with_shelter = np.append(x, shelter_x)
    y_with_shelter = np.append(y, shelter_y)
    
    #find the min, max and range in the whole dataset for normalization
    combined_min = y_with_shelter.min()
    combined_max = y_with_shelter.max()
    combined_range = combined_max - combined_min    
    
    #normalize all values 
    x_norm = (x - combined_min) / combined_range
    y_norm = (y - combined_min) / combined_range

    shelter_normalized_x = (shelter_x - combined_min) / combined_range
    shelter_normalized_y = (shelter_y - combined_min) / combined_range
    
    shelter_norm_x = shelter_normalized_x - x_norm[0]
    shelter_norm_y = shelter_normalized_y - y_norm[0]
    
    normalized['x'] = x_norm - x_norm[0]
    normalized['y'] = y_norm - y_norm[0]
    
    
    return normalized, shelter_norm_x, shelter_norm_y, combined_range